# Importing Libraries

In [5]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline

# Importing Data

In [6]:
!unzip "/content/drive/MyDrive/Copy of file_1.zip"

Archive:  /content/drive/MyDrive/Copy of file_1.zip
  inflating: file_1.hdf5             


In [7]:
import h5py
hdf5_path = "/content/file_1.hdf5"
dataset = h5py.File(hdf5_path, 'r')

In [8]:
dataset.keys()

<KeysViewHDF5 ['H_Im', 'H_Re', 'Pos', 'SNR']>

# Data Processing

In [9]:
data1 = dataset['H_Im']
data2 = dataset['H_Re']
print(data1.shape)
print(data2.shape)

(512, 56, 924, 5)
(512, 56, 924, 5)


In [10]:
data3 = dataset['Pos']
data4 = dataset['SNR']
print(data3.shape)
print(data4.shape)

(512, 3)
(512, 56, 5)


In [11]:
# Converting data into tensors and creating amplitude from real and imaginary data
data1 = tf.constant(data1)
data2 = tf.constant(data2)
data = tf.math.sqrt(tf.add(tf.math.square(data1), tf.math.square(data2)))

In [12]:
data

<tf.Tensor: shape=(512, 56, 924, 5), dtype=float16, numpy=
array([[[[0.1469  , 0.1517  , 0.1512  , 0.1583  , 0.1528  ],
         [0.2039  , 0.211   , 0.2098  , 0.2169  , 0.212   ],
         [0.2477  , 0.2551  , 0.253   , 0.2595  , 0.256   ],
         ...,
         [0.3643  , 0.3467  , 0.363   , 0.3489  , 0.3545  ],
         [0.3098  , 0.2961  , 0.3113  , 0.2957  , 0.3027  ],
         [0.2291  , 0.2225  , 0.2336  , 0.2195  , 0.2264  ]],

        [[0.2854  , 0.295   , 0.2979  , 0.2952  , 0.295   ],
         [0.3965  , 0.4087  , 0.4133  , 0.4094  , 0.4116  ],
         [0.4797  , 0.4932  , 0.499   , 0.4949  , 0.4988  ],
         ...,
         [0.4036  , 0.4058  , 0.4067  , 0.3997  , 0.3965  ],
         [0.336   , 0.3438  , 0.3398  , 0.3323  , 0.3337  ],
         [0.2444  , 0.255   , 0.2473  , 0.2412  , 0.2467  ]],

        [[0.      , 0.      , 0.872   , 0.896   , 0.914   ],
         [0.      , 0.      , 1.1875  , 1.226   , 1.256   ],
         [0.      , 0.      , 1.43    , 1.467   , 1.507

In [13]:
# Applying Median Filter
tfa.image.median_filter2d(data, filter_shape=3 , padding='REFLECT')

<tf.Tensor: shape=(512, 56, 924, 5), dtype=float16, numpy=
array([[[[0.2854  , 0.295   , 0.2979  , 0.2952  , 0.295   ],
         [0.2854  , 0.295   , 0.2979  , 0.2952  , 0.295   ],
         [0.3965  , 0.4087  , 0.4133  , 0.4094  , 0.4116  ],
         ...,
         [0.3872  , 0.371   , 0.3855  , 0.3743  , 0.3777  ],
         [0.336   , 0.3438  , 0.3398  , 0.3323  , 0.3337  ],
         [0.3098  , 0.2961  , 0.3113  , 0.2957  , 0.3027  ]],

        [[0.2039  , 0.211   , 0.4133  , 0.4094  , 0.4116  ],
         [0.2039  , 0.211   , 0.4133  , 0.4094  , 0.4116  ],
         [0.2477  , 0.2551  , 0.499   , 0.4949  , 0.4988  ],
         ...,
         [0.336   , 0.3438  , 0.3645  , 0.3489  , 0.3777  ],
         [0.2444  , 0.255   , 0.3113  , 0.2957  , 0.326   ],
         [0.2444  , 0.255   , 0.299   , 0.2864  , 0.3027  ]],

        [[0.2854  , 0.295   , 0.4133  , 0.4094  , 0.4116  ],
         [0.2854  , 0.295   , 0.4683  , 0.4678  , 0.4675  ],
         [0.3965  , 0.4087  , 0.5103  , 0.5107  , 0.51 

In [30]:
# Normalizing the data
Ai = tf.math.reduce_sum(data, [0, 2, 3])
maxAi = tf.math.reduce_max(Ai)
minAi = tf.math.reduce_min(Ai)
Ai = tf.constant(Ai.numpy() - minAi.numpy())/(maxAi.numpy() - minAi.numpy())

In [31]:
Ai

<tf.Tensor: shape=(56,), dtype=float16, numpy=
array([0.2617  , 0.3333  , 1.      , 0.1351  , 0.3333  , 0.3333  ,
       0.3333  , 0.      , 0.3333  , 0.262   , 0.3333  , 0.3333  ,
       0.3333  , 0.3333  , 0.3333  , 0.3333  , 0.3333  , 0.3333  ,
       0.3333  , 0.3333  , 0.02344 , 0.3333  , 0.3333  , 0.3333  ,
       0.3347  , 0.3333  , 0.234   , 0.1445  , 0.3333  , 0.03354 ,
       0.3333  , 0.5234  , 0.3333  , 0.3333  , 0.012695, 0.338   ,
       0.3333  , 0.      , 0.3333  , 0.3333  , 0.2598  , 0.3333  ,
       0.3333  , 0.03125 , 0.3333  , 0.3333  , 0.3333  , 0.3333  ,
       0.3333  , 0.3333  , 0.3333  , 0.001628, 0.3333  , 0.3333  ,
       0.3333  , 0.3333  ], dtype=float16)>